# Classification mnist

This is an example to show how you can use MapReader on non-patchified datasets.

# Load
https://mapreader.readthedocs.io/en/latest/using-mapreader/step-by-step-guide/2-load.html

## Load images

We will load our mnist images as patches so MapReader knows we are going to be using these as our unit of analysis.

In [ ]:
from mapreader import load_patches

my_files = load_patches("./small_mnist/*.png")

In [ ]:
# len() shows the total number of images currently read (or sliced, see below)
print(f"Number of images: {len(my_files)}")

In [ ]:
print(my_files)

In [ ]:
my_files.show_sample(num_samples=3, tree_level="patch")

In [ ]:
my_files.add_shape(tree_level="patch")

In [7]:
parent_df, patch_df = my_files.convert_images()

In [ ]:
# add pixel bounds from shape of the patch (min_x, min_y, max_x, max_y)
patch_df["pixel_bounds"] = patch_df["shape"].apply(lambda x: (0, 0, x[1], x[0]))
patch_df.head()

# Annotate
https://mapreader.readthedocs.io/en/latest/using-mapreader/step-by-step-guide/3-annotate.html

In [10]:
from mapreader import Annotator

In [11]:
annotator = Annotator(
    patch_df=patch_df,
    parent_df=parent_df,
    labels=["1", "3"],
    annotations_dir="./annotations_mnist",
    username="kasra",
    task_name="mnist",
    show_context=False,
)

In [ ]:
annotator.annotate(resize_to=400)

# Classify - Train

https://mapreader.readthedocs.io/en/latest/using-mapreader/step-by-step-guide/4-classify/index.html

## Read annotations

In [13]:
from mapreader import AnnotationsLoader

In [ ]:
annotated_images = AnnotationsLoader()

annotated_images.load("./annotations_mnist/mnist_#kasra#.csv", images_dir="./small_mnist/")

In [ ]:
# show sample images for one label (label_to_show)
annotated_images.show_sample(label_to_show="1", num_samples=6)

In [ ]:
# show an image based on its index
annotated_images.show_patch(patch_id="20989.png")

## Prepare datasets and dataloaders

In [ ]:
annotated_images.create_datasets(frac_train=0.7, frac_val=0.2, frac_test=0.1)

In [ ]:
dataloaders = annotated_images.create_dataloaders(batch_size=8, sampler="default")

## Set up `ClassifierContainer`

### Load a (pretrained) PyTorch model and combine with dataloaders

In [19]:
from mapreader import ClassifierContainer

In [ ]:
my_classifier = ClassifierContainer(model="resnet18", 
                                    labels_map={0: "3", 1: "1"},
                                    dataloaders=dataloaders
                                    )

In [ ]:
my_classifier.model_summary()

### Set up loss function, optimizer and scheduler

In [ ]:
my_classifier.add_loss_fn("cross-entropy")

In [23]:
params_to_optimize = my_classifier.generate_layerwise_lrs(
    min_lr=1e-4, max_lr=1e-3, spacing="geomspace"
)

In [24]:
my_classifier.initialize_optimizer(params2optimize=params_to_optimize)

In [25]:
my_classifier.initialize_scheduler()

## Train/fine-tune a model

In [ ]:
my_classifier.train(
    num_epochs=15,
    save_model_dir="./models_mnist",
    tensorboard_path="tboard_mnist",
    tmp_file_save_freq=2,
    remove_after_load=False,
    print_info_batch_freq=5,
)

In [ ]:
my_classifier.list_metrics()

In [ ]:
my_classifier.plot_metric(
    metrics=["loss"],
)

In [ ]:
my_classifier.plot_metric(
    metrics=['precision_macro', 'recall_macro','fscore_macro','support_macro','rocauc_macro'],
)

# Classify - Infer

https://mapreader.readthedocs.io/en/latest/using-mapreader/step-by-step-guide/4-classify/index.html

## Create dataset with all mnist data and add to ``ClassifierContainer``

In [30]:
from mapreader import PatchDataset

In [31]:
data = PatchDataset(patch_df, transform="val")

In [32]:
my_classifier.load_dataset(data, set_name="all_mnist")

## Infer labels

In [ ]:
my_classifier.inference("all_mnist")

In [ ]:
my_classifier.show_inference_sample_results(
    label="3", set_name="all_mnist", min_conf=99
)